In [37]:
## 데이터 분석 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import re
import time
import random
from tqdm.notebook import tqdm

## Selenium 관련 환경설정
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
caps = DesiredCapabilities.CHROME
caps["pageLoadStrategy"] = "none"

## bs 라이브러리
from bs4 import BeautifulSoup
import requests

# 페이지 로드 간소화 라이브러리
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

# # webdriver options
options = wb.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
# options.add_argument('disable-dev-shm-usage')

# 이미지 로딩 비활성화
options.add_argument('--disable-images')
options.add_experimental_option("prefs", {
    'profile.managed_default_content_settings.images': 2,
    'profile.default_content_setting_values.media_stream_mic': 2,
    'profile.default_content_setting_values.media_stream_camera': 2,
    'profile.default_content_setting_values.geolocation': 2,
    'profile.default_content_setting_values.notifications': 2,
    'profile.default_content_setting_values.media_stream': 2,
    'profile.default_content_setting_values.automatic_downloads': 2,
    'profile.default_content_setting_values.autoplay': 2  # 자동재생 비활성화
})
options.add_argument('--blink-settings=imagesEnabled=false')
options.add_argument('--mute-audio')  # 소리 비활성화

In [98]:
origin_df = pd.read_csv('melon_full_data.csv')
origin_df.head(5)

,albumCov,title,artistIdx,albumIdx,genreIdx,releasedAt,lyrics,year
0,https://cdnimg.melon.co.kr/cm2/album/images/10...,Until I Found You,Stephen Sanchez,Until I Found You,포크,2021.09.01,"Georgia, Wrap me up in all your...\r\nI want y...",2021
1,https://cdnimg.melon.co.kr/cm2/album/images/11...,IT'S YOU (feat. keshi),MAX,LOVE IN STEREO,POP,2024.02.16,"Mm-hmm, no-oh, yeah, yeah\r\n\r\nIt's you\r\nL...",2024
2,https://cdnimg.melon.co.kr/cm2/album/images/10...,UNDERSTAND,keshi,GABRIEL (Korean Special Edition),POP,2022.03.25,Never thought that I'd find\r\nThat the one in...,2022
3,https://cdnimg.melon.co.kr/cm2/album/images/10...,stars,Arash Buana,stars,POP,2021.02.28,NaN,2021
4,https://cdnimg.melon.co.kr/cm2/album/images/11...,Till There Was You,Imaginary Future,Till There Was You,POP,2023.05.02,NaN,2023


In [99]:
origin_df['song'] = origin_df['title'] + " " + origin_df['artistIdx']
origin_df

,albumCov,title,artistIdx,albumIdx,genreIdx,releasedAt,lyrics,year,song
0,https://cdnimg.melon.co.kr/cm2/album/images/10...,Until I Found You,Stephen Sanchez,Until I Found You,포크,2021.09.01,"Georgia, Wrap me up in all your...\r\nI want y...",2021,Until I Found You Stephen Sanchez
1,https://cdnimg.melon.co.kr/cm2/album/images/11...,IT'S YOU (feat. keshi),MAX,LOVE IN STEREO,POP,2024.02.16,"Mm-hmm, no-oh, yeah, yeah\r\n\r\nIt's you\r\nL...",2024,IT'S YOU (feat. keshi) MAX
2,https://cdnimg.melon.co.kr/cm2/album/images/10...,UNDERSTAND,keshi,GABRIEL (Korean Special Edition),POP,2022.03.25,Never thought that I'd find\r\nThat the one in...,2022,UNDERSTAND keshi
3,https://cdnimg.melon.co.kr/cm2/album/images/10...,stars,Arash Buana,stars,POP,2021.02.28,NaN,2021,stars Arash Buana
4,https://cdnimg.melon.co.kr/cm2/album/images/11...,Till There Was You,Imaginary Future,Till There Was You,POP,2023.05.02,NaN,2023,Till There Was You Imaginary Future
...,...,...,...,...,...,...,...,...,...
18647,https://cdnimg.melon.co.kr/cm2/album/images/10...,찌질이,안재우,나쁜년,"R&B/Soul, 인디음악",2021.01.05,그냥 그때 돌아서지 말걸\r\n라고 뱉으며 나 후회는\r\n한 백번 천번쯤에\r\n...,2021,찌질이 안재우
18648,https://cdnimg.melon.co.kr/cm2/album/images/10...,She Said (with BIBI),Crush,with HER,랩/힙합,2020.10.20,청소년 보호법에 따라 성인 인증이 필요한 콘텐츠 입니다. 성인 인증 후 이용해 주세...,2020,She Said (with BIBI) Crush
18649,https://cdnimg.melon.co.kr/cm2/album/images/10...,Do It For Love (Feat. 죠지),THAMA,Do It For Love,R&B/Soul,2020.06.24,아침햇살은 밝아\r\n텅 빈 하루를 살아 난\r\n나이는 적지 않아\r\n가끔씩 궁...,2020,Do It For Love (Feat. 죠지) THAMA
18650,https://cdnimg.melon.co.kr/cm2/album/images/10...,몇번째 (Song By 찬현),KozyPop,Seoul Vibes Pt.15,"R&B/Soul, 인디음악",2020.04.15,I love u 근데 나는 너무 아퍼\r\n내 맘을 아는 거면 너는 진짜 나뻐\r\...,2020,몇번째 (Song By 찬현) KozyPop


In [40]:
song_list = df['song'].tolist()

In [5]:
song_list

['Until I Found You Stephen Sanchez',
 "IT'S YOU (feat. keshi) MAX",
 'UNDERSTAND keshi',
 'stars Arash Buana',
 'Till There Was You Imaginary Future',
 'Mozart : The Magic Flute K.620 Act.II - Queen Of The Night Aria \'Der Holle Rache\' (모차르트 : 마술 피리 - 2막. 밤의 여왕의 아리아) (From "Amadeus") Bulgarian National Radio Symphony Orchestra, John Landor, Rachel Rosales',
 'What We Found Jesse Barrera, Nieman, Melissa Polinar',
 'Real Love Still Exists 헨리 (HENRY), Yuna',
 'Old Love (Stripped) yuji, putri dahlia',
 'Letters (Acoustic) Maximillian',
 'I Want You Stephen Sanchez',
 'Butterflies (Acoustic) Abe Parker',
 'EDIE CELINE (Stripped Version) MAX',
 'Ripples (Acoustic) Maximillian',
 'still that kid (deep down) Yaeow',
 'favorite lesson Yaeow',
 'This Is How It Feels d4vd, Laufey',
 'The Green (Acoustic ver.) Chris James',
 'Stuck Here Without You Yaeow',
 'What I Feel Is Love Etham',
 'all i need (the distance song) Avery Lynch',
 'Everyone Adores You (quiet) Matt Maltese',
 'Starlight 92914'

In [6]:
song_unique_list = df['song'].unique().tolist()
len(song_unique_list)

11081

In [18]:
song_df = pd.DataFrame(song_unique_list)
song_df.head(3)

,0
0,Until I Found You Stephen Sanchez
1,IT'S YOU (feat. keshi) MAX
2,UNDERSTAND keshi


In [8]:
# 리스트 길이를 계산하고 반으로 나눔
mid_index = len(song_unique_list) // 2

# 리스트를 두 개로 분할
first_half = song_unique_list[:mid_index]
second_half = song_unique_list[mid_index:]

In [84]:
driver = wb.Chrome()
    # YouTube 플레이리스트 URL로 이동
driver.get('https://www.youtube.com/')

In [85]:
search = driver.find_element("name", "search_query")

In [86]:
search.clear()
search.send_keys('Until I Found You Stephen Sanchez')

In [87]:
search.send_keys(Keys.ENTER)
time.sleep(2)

In [89]:
link = driver.find_element(By.CSS_SELECTOR, "#video-title")
link.get_attribute('href')

'https://www.youtube.com/watch?v=GxldQ9eX2wo&pp=ygUhVW50aWwgSSBGb3VuZCBZb3UgU3RlcGhlbiBTYW5jaGV6'

In [52]:
btn = driver.find_element(By.CSS_SELECTOR, "#thumbnail > yt-image > img")
btn.click()

In [54]:
btn = driver.find_element(By.CSS_SELECTOR, "#top-level-buttons-computed > yt-button-view-model > button-view-model > button > yt-touch-feedback-shape > div > div.yt-spec-touch-feedback-shape__fill")
btn.click()

In [35]:
btn = driver.find_element(By.CSS_SELECTOR, "#target > yt-icon > yt-icon-shape > icon-shape > div")
btn.click()

In [37]:
value = driver.find_element(By.CSS_SELECTOR, "#textarea")

In [38]:
textarea_value = value.get_attribute('value')

In [39]:
youtube_iframe.append(textarea_value)

'<iframe width="560" height="315" src="https://www.youtube.com/embed/GxldQ9eX2wo?si=cr8aZICNYzwksbR0" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>'

In [40]:
btn = driver.find_element(By.CSS_SELECTOR, "#close-panel-icon > yt-icon-shape > icon-shape > div")
btn.click()

In [11]:
driver = wb.Chrome(options=options)
    # YouTube 플레이리스트 URL로 이동
driver.get('https://www.youtube.com/')
youtube_iframe1 = []

for song in tqdm(first_half):
    try:
        # 검색 창 찾기 및 노래 검색
        search = driver.find_element(By.NAME, "search_query")
        search.clear()
        search.send_keys(song)
        search.send_keys(Keys.ENTER)
        
        # 랜덤으로 대기 시간을 설정
        time.sleep(random.uniform(3, 4))
        
        # 동영상 클릭
        btn = driver.find_element(By.CSS_SELECTOR, "#video-title")
        value =btn.get_attribute('href')

        youtube_iframe1.append(value)
    except Exception as e:
        print(f"Error processing '{song}': {e}")
        youtube_iframe1.append(None)
    
driver.quit()


  0%|          | 0/5540 [00:00<?, ?it/s]

Error processing 'Mozart : The Magic Flute K.620 Act.II - Queen Of The Night Aria 'Der Holle Rache' (모차르트 : 마술 피리 - 2막. 밤의 여왕의 아리아) (From "Amadeus") Bulgarian National Radio Symphony Orchestra, John Landor, Rachel Rosales': Message: no such element: Unable to locate element: {"method":"css selector","selector":"#video-title"}
  (Session info: chrome-headless-shell=125.0.6422.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6203B1F52+60322]
	(No symbol) [0x00007FF62032CEC9]
	(No symbol) [0x00007FF6201E7EBA]
	(No symbol) [0x00007FF620237676]
	(No symbol) [0x00007FF62023773C]
	(No symbol) [0x00007FF62027E967]
	(No symbol) [0x00007FF62025C25F]
	(No symbol) [0x00007FF62027BC80]
	(No symbol) [0x00007FF62025BFC3]
	(No symbol) [0x00007FF620229617]
	(No symbol) [0x00007FF62022A211]
	GetHandleVerifier [0x00007FF6206C94AD+3301629]
	GetHandleVerifier [0x000

In [14]:
iframe_df1 = pd.DataFrame(youtube_iframe1, columns=['iframe'])
iframe_df1.to_csv('유튜브iframe정보1.csv',index=False,encoding='utf-8-sig')

In [9]:
driver = wb.Chrome(options=options)
    # YouTube 플레이리스트 URL로 이동
driver.get('https://www.youtube.com/')
youtube_iframe2 = []

for song in tqdm(second_half):
    try:
        # 검색 창 찾기 및 노래 검색
        search = driver.find_element(By.NAME, "search_query")
        search.clear()
        search.send_keys(song)
        search.send_keys(Keys.ENTER)
        
        # 랜덤으로 대기 시간을 설정
        time.sleep(random.uniform(3, 4))
        
        # 동영상 클릭
        btn = driver.find_element(By.CSS_SELECTOR, "#video-title")
        value =btn.get_attribute('href')

        youtube_iframe2.append(value)
    except Exception as e:
        print(f"Error processing '{song}': {e}")
        youtube_iframe2.append(None)
    
driver.quit()


  0%|          | 0/5541 [00:00<?, ?it/s]

Error processing 'We're All Gonna Die Olivia O'Brien': Message: no such element: Unable to locate element: {"method":"css selector","selector":"#video-title"}
  (Session info: chrome-headless-shell=125.0.6422.144); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF77C2A1F52+60322]
	(No symbol) [0x00007FF77C21CEC9]
	(No symbol) [0x00007FF77C0D7EBA]
	(No symbol) [0x00007FF77C127676]
	(No symbol) [0x00007FF77C12773C]
	(No symbol) [0x00007FF77C16E967]
	(No symbol) [0x00007FF77C14C25F]
	(No symbol) [0x00007FF77C16BC80]
	(No symbol) [0x00007FF77C14BFC3]
	(No symbol) [0x00007FF77C119617]
	(No symbol) [0x00007FF77C11A211]
	GetHandleVerifier [0x00007FF77C5B94AD+3301629]
	GetHandleVerifier [0x00007FF77C6036D3+3605283]
	GetHandleVerifier [0x00007FF77C5F9450+3563680]
	GetHandleVerifier [0x00007FF77C354326+790390]
	(No symbol) [0x00007FF77C22750F]
	(No symbol) [0x0

In [11]:
iframe_df2 = pd.DataFrame(youtube_iframe2, columns=['iframe'])
iframe_df2.to_csv('유튜브iframe정보2.csv',index=False,encoding='utf-8-sig')

In [94]:
df1 = pd.read_csv('유튜브iframe정보1.csv')
df2 = pd.read_csv('유튜브iframe정보2.csv')

FileNotFoundError: [Errno 2] No such file or directory: '유튜브iframe정보1.csv'

In [95]:
df1.to_csv('유튜브iframe정보1.csv',index=False,encoding='utf-8-sig')
df2.to_csv('유튜브iframe정보2.csv',index=False,encoding='utf-8-sig')

In [96]:
va = 'https://www.youtube.com/watch?v=FHlcOxjA-qM&pp=ygUQ7IKs7J20IOq5gOyVhOumhA%3D%3D'
filtered_dfdf = df1[df1['iframe'] == va]
filtered_dfdf

,iframe
3588,https://www.youtube.com/watch?v=FHlcOxjA-qM&pp...


In [107]:
pull_df = pd.concat([df1, df2])
pull_df

,iframe
0,https://www.youtube.com/watch?v=GxldQ9eX2wo&pp...
1,https://www.youtube.com/watch?v=zWxuc8L0W_Y&pp...
2,https://www.youtube.com/watch?v=6DcUnqZqTvI&pp...
3,https://www.youtube.com/watch?v=SfrIHYu8u9c&pp...
4,https://www.youtube.com/watch?v=RI4CLyFhUAg&pp...
...,...
5536,https://www.youtube.com/watch?v=FVftpgsfw1o&pp...
5537,https://www.youtube.com/watch?v=SMJ3eTwzJKg&pp...
5538,https://www.youtube.com/watch?v=8yo9GCnM3jc&pp...
5539,https://www.youtube.com/watch?v=21aFacRNoWE&pp...


In [113]:
song_df = pd.DataFrame(origin_df['song'].unique().tolist(),columns=['song'])
song_df

,song
0,Until I Found You Stephen Sanchez
1,IT'S YOU (feat. keshi) MAX
2,UNDERSTAND keshi
3,stars Arash Buana
4,Till There Was You Imaginary Future
...,...
11076,"밤하늘의 저 별처럼 헤이즈 (Heize), 펀치 (Punch)"
11077,SOON (Feat. BewhY) 다이나믹 듀오
11078,She Said (with BIBI) Crush
11079,Do It For Love (Feat. 죠지) THAMA


In [116]:
specific_song = '사이 김아름'
filtered_df = song_df[song_df['song'] == specific_song]
filtered_df

,song
3588,사이 김아름


In [117]:
specific_value = 'FHlcOxjA-qM'
filtered_df = dadada[dadada['youtube'] == specific_value]
filtered_df

,youtube
3588,FHlcOxjA-qM


In [118]:
# 인덱스 초기화
song_df = song_df.reset_index(drop=True)
pull_df = pull_df.reset_index(drop=True)

In [119]:
youtube_df = pd.concat([song_df, pull_df], axis=1)
youtube_df

,song,iframe
0,Until I Found You Stephen Sanchez,https://www.youtube.com/watch?v=GxldQ9eX2wo&pp...
1,IT'S YOU (feat. keshi) MAX,https://www.youtube.com/watch?v=zWxuc8L0W_Y&pp...
2,UNDERSTAND keshi,https://www.youtube.com/watch?v=6DcUnqZqTvI&pp...
3,stars Arash Buana,https://www.youtube.com/watch?v=SfrIHYu8u9c&pp...
4,Till There Was You Imaginary Future,https://www.youtube.com/watch?v=RI4CLyFhUAg&pp...
...,...,...
11076,"밤하늘의 저 별처럼 헤이즈 (Heize), 펀치 (Punch)",https://www.youtube.com/watch?v=FVftpgsfw1o&pp...
11077,SOON (Feat. BewhY) 다이나믹 듀오,https://www.youtube.com/watch?v=SMJ3eTwzJKg&pp...
11078,She Said (with BIBI) Crush,https://www.youtube.com/watch?v=8yo9GCnM3jc&pp...
11079,Do It For Love (Feat. 죠지) THAMA,https://www.youtube.com/watch?v=21aFacRNoWE&pp...


In [120]:
spe_song = '사이 김아름'
fil_df = youtube_df[youtube_df['song'] == spe_song]
fil_df

,song,iframe
3588,사이 김아름,https://www.youtube.com/watch?v=FHlcOxjA-qM&pp...


In [41]:
youtube_df.to_csv('유튜브iframe정보_pull.csv',index=False,encoding='utf-8-sig')
youtube_df.head(2)

,0,iframe
0,Until I Found You Stephen Sanchez,https://www.youtube.com/watch?v=GxldQ9eX2wo&pp...
1,IT'S YOU (feat. keshi) MAX,https://www.youtube.com/watch?v=zWxuc8L0W_Y&pp...


In [126]:
song_iframe_dict = youtube_df.set_index('song')['iframe'].to_dict()
song_iframe_dict

{'Until I Found You Stephen Sanchez': 'https://www.youtube.com/watch?v=GxldQ9eX2wo&pp=ygUhVW50aWwgSSBGb3VuZCBZb3UgU3RlcGhlbiBTYW5jaGV6',
 "IT'S YOU (feat. keshi) MAX": 'https://www.youtube.com/watch?v=zWxuc8L0W_Y&pp=ygUaSVQnUyBZT1UgKGZlYXQuIGtlc2hpKSBNQVg%3D',
 'UNDERSTAND keshi': 'https://www.youtube.com/watch?v=6DcUnqZqTvI&pp=ygUQVU5ERVJTVEFORCBrZXNoaQ%3D%3D',
 'stars Arash Buana': 'https://www.youtube.com/watch?v=SfrIHYu8u9c&pp=ygURc3RhcnMgQXJhc2ggQnVhbmE%3D',
 'Till There Was You Imaginary Future': 'https://www.youtube.com/watch?v=RI4CLyFhUAg&pp=ygUjVGlsbCBUaGVyZSBXYXMgWW91IEltYWdpbmFyeSBGdXR1cmU%3D',
 'Mozart : The Magic Flute K.620 Act.II - Queen Of The Night Aria \'Der Holle Rache\' (모차르트 : 마술 피리 - 2막. 밤의 여왕의 아리아) (From "Amadeus") Bulgarian National Radio Symphony Orchestra, John Landor, Rachel Rosales': nan,
 'What We Found Jesse Barrera, Nieman, Melissa Polinar': 'https://www.youtube.com/watch?v=uKwtHk4Ok7E&pp=ygU0V2hhdCBXZSBGb3VuZCBKZXNzZSBCYXJyZXJhLCBOaWVtYW4sIE1lbGlzc2EgUG9

In [128]:
origin_df['youtube'] = origin_df['song']
origin_df.head(10)

,albumCov,title,artistIdx,albumIdx,genreIdx,releasedAt,lyrics,year,song,youtube
0,https://cdnimg.melon.co.kr/cm2/album/images/10...,Until I Found You,Stephen Sanchez,Until I Found You,포크,2021.09.01,"Georgia, Wrap me up in all your...\r\nI want y...",2021,Until I Found You Stephen Sanchez,Until I Found You Stephen Sanchez
1,https://cdnimg.melon.co.kr/cm2/album/images/11...,IT'S YOU (feat. keshi),MAX,LOVE IN STEREO,POP,2024.02.16,"Mm-hmm, no-oh, yeah, yeah\r\n\r\nIt's you\r\nL...",2024,IT'S YOU (feat. keshi) MAX,IT'S YOU (feat. keshi) MAX
2,https://cdnimg.melon.co.kr/cm2/album/images/10...,UNDERSTAND,keshi,GABRIEL (Korean Special Edition),POP,2022.03.25,Never thought that I'd find\r\nThat the one in...,2022,UNDERSTAND keshi,UNDERSTAND keshi
3,https://cdnimg.melon.co.kr/cm2/album/images/10...,stars,Arash Buana,stars,POP,2021.02.28,NaN,2021,stars Arash Buana,stars Arash Buana
4,https://cdnimg.melon.co.kr/cm2/album/images/11...,Till There Was You,Imaginary Future,Till There Was You,POP,2023.05.02,NaN,2023,Till There Was You Imaginary Future,Till There Was You Imaginary Future
5,https://cdnimg.melon.co.kr/cm2/album/images/10...,Mozart : The Magic Flute K.620 Act.II - Queen ...,"Bulgarian National Radio Symphony Orchestra, J...",Mozart in Film,"클래식, 교향/관현악",2020.12.25,NaN,2020,Mozart : The Magic Flute K.620 Act.II - Queen ...,Mozart : The Magic Flute K.620 Act.II - Queen ...
6,https://cdnimg.melon.co.kr/cm2/album/images/10...,What We Found,"Jesse Barrera, Nieman, Melissa Polinar",What We Found,R&B/Soul,2021.02.19,Every page worn in\r\nAnd the ink has faded\r\...,2021,"What We Found Jesse Barrera, Nieman, Melissa P...","What We Found Jesse Barrera, Nieman, Melissa P..."
7,https://cdnimg.melon.co.kr/cm2/album/images/11...,Real Love Still Exists,"헨리 (HENRY), Yuna",Real Love Still Exists,발라드,2023.03.08,What you see when you look at me\r\nAn image o...,2023,"Real Love Still Exists 헨리 (HENRY), Yuna","Real Love Still Exists 헨리 (HENRY), Yuna"
8,https://cdnimg.melon.co.kr/cm2/album/images/11...,Old Love (Stripped),"yuji, putri dahlia",Old Love (Stripped),POP,2022.10.20,NaN,2022,"Old Love (Stripped) yuji, putri dahlia","Old Love (Stripped) yuji, putri dahlia"
9,https://cdnimg.melon.co.kr/cm2/album/images/11...,Letters (Acoustic),Maximillian,Unplugged,POP,2023.06.09,Same place but something's different\r\nYeah I...,2023,Letters (Acoustic) Maximillian,Letters (Acoustic) Maximillian


In [130]:
origin_df['youtube'] = origin_df['youtube'].map(song_iframe_dict)
origin_df

,albumCov,title,artistIdx,albumIdx,genreIdx,releasedAt,lyrics,year,song,youtube
0,https://cdnimg.melon.co.kr/cm2/album/images/10...,Until I Found You,Stephen Sanchez,Until I Found You,포크,2021.09.01,"Georgia, Wrap me up in all your...\r\nI want y...",2021,Until I Found You Stephen Sanchez,https://www.youtube.com/watch?v=GxldQ9eX2wo&pp...
1,https://cdnimg.melon.co.kr/cm2/album/images/11...,IT'S YOU (feat. keshi),MAX,LOVE IN STEREO,POP,2024.02.16,"Mm-hmm, no-oh, yeah, yeah\r\n\r\nIt's you\r\nL...",2024,IT'S YOU (feat. keshi) MAX,https://www.youtube.com/watch?v=zWxuc8L0W_Y&pp...
2,https://cdnimg.melon.co.kr/cm2/album/images/10...,UNDERSTAND,keshi,GABRIEL (Korean Special Edition),POP,2022.03.25,Never thought that I'd find\r\nThat the one in...,2022,UNDERSTAND keshi,https://www.youtube.com/watch?v=6DcUnqZqTvI&pp...
3,https://cdnimg.melon.co.kr/cm2/album/images/10...,stars,Arash Buana,stars,POP,2021.02.28,NaN,2021,stars Arash Buana,https://www.youtube.com/watch?v=SfrIHYu8u9c&pp...
4,https://cdnimg.melon.co.kr/cm2/album/images/11...,Till There Was You,Imaginary Future,Till There Was You,POP,2023.05.02,NaN,2023,Till There Was You Imaginary Future,https://www.youtube.com/watch?v=RI4CLyFhUAg&pp...
...,...,...,...,...,...,...,...,...,...,...
18647,https://cdnimg.melon.co.kr/cm2/album/images/10...,찌질이,안재우,나쁜년,"R&B/Soul, 인디음악",2021.01.05,그냥 그때 돌아서지 말걸\r\n라고 뱉으며 나 후회는\r\n한 백번 천번쯤에\r\n...,2021,찌질이 안재우,https://www.youtube.com/watch?v=Vs9QtN8fvho&pp...
18648,https://cdnimg.melon.co.kr/cm2/album/images/10...,She Said (with BIBI),Crush,with HER,랩/힙합,2020.10.20,청소년 보호법에 따라 성인 인증이 필요한 콘텐츠 입니다. 성인 인증 후 이용해 주세...,2020,She Said (with BIBI) Crush,https://www.youtube.com/watch?v=8yo9GCnM3jc&pp...
18649,https://cdnimg.melon.co.kr/cm2/album/images/10...,Do It For Love (Feat. 죠지),THAMA,Do It For Love,R&B/Soul,2020.06.24,아침햇살은 밝아\r\n텅 빈 하루를 살아 난\r\n나이는 적지 않아\r\n가끔씩 궁...,2020,Do It For Love (Feat. 죠지) THAMA,https://www.youtube.com/watch?v=21aFacRNoWE&pp...
18650,https://cdnimg.melon.co.kr/cm2/album/images/10...,몇번째 (Song By 찬현),KozyPop,Seoul Vibes Pt.15,"R&B/Soul, 인디음악",2020.04.15,I love u 근데 나는 너무 아퍼\r\n내 맘을 아는 거면 너는 진짜 나뻐\r\...,2020,몇번째 (Song By 찬현) KozyPop,https://www.youtube.com/watch?v=xYohdxG_izE&pp...


In [131]:
origin_df['youtube'].to_csv('iframe_full.csv',index=False,encoding='utf-8-sig')

In [132]:
you_df = pd.read_csv('iframe_full.csv')
you_df

,youtube
0,https://www.youtube.com/watch?v=GxldQ9eX2wo&pp...
1,https://www.youtube.com/watch?v=zWxuc8L0W_Y&pp...
2,https://www.youtube.com/watch?v=6DcUnqZqTvI&pp...
3,https://www.youtube.com/watch?v=SfrIHYu8u9c&pp...
4,https://www.youtube.com/watch?v=RI4CLyFhUAg&pp...
...,...
18647,https://www.youtube.com/watch?v=Vs9QtN8fvho&pp...
18648,https://www.youtube.com/watch?v=8yo9GCnM3jc&pp...
18649,https://www.youtube.com/watch?v=21aFacRNoWE&pp...
18650,https://www.youtube.com/watch?v=xYohdxG_izE&pp...


In [57]:
you_df['youtube'][0]

'https://www.youtube.com/watch?v=yFPZkd9pPG8&pp=ygUbUEFDS0lUVVAhIChwcm9kLiBCTVRKKSBwSC0x'

In [133]:
# 비디오 ID를 추출하는 함수
def extract_video_id(url):
    pattern = r"v=([a-zA-Z0-9_-]{11})"
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    else:
        return None

In [134]:
you_df['youtube'] = you_df['youtube'].str.extract(r"v=([a-zA-Z0-9_-]{11})")[0]
you_df

,youtube
0,GxldQ9eX2wo
1,zWxuc8L0W_Y
2,6DcUnqZqTvI
3,SfrIHYu8u9c
4,RI4CLyFhUAg
...,...
18647,Vs9QtN8fvho
18648,8yo9GCnM3jc
18649,21aFacRNoWE
18650,xYohdxG_izE


In [135]:
you_df['youtube'].to_csv('iframe_link_full.csv',index=False,encoding='utf-8-sig')

In [136]:
total_nan = you_df.isnull().sum().sum()
print(f"Total NaN values: {total_nan}")

Total NaN values: 138


In [137]:
you_df = you_df.fillna('-')
you_df

,youtube
0,GxldQ9eX2wo
1,zWxuc8L0W_Y
2,6DcUnqZqTvI
3,SfrIHYu8u9c
4,RI4CLyFhUAg
...,...
18647,Vs9QtN8fvho
18648,8yo9GCnM3jc
18649,21aFacRNoWE
18650,xYohdxG_izE


In [140]:
you_df.to_csv('iframe_link_full_전처리.csv',index=False,encoding='utf-8-sig')

In [145]:
you_df

,youtube
0,GxldQ9eX2wo
1,zWxuc8L0W_Y
2,6DcUnqZqTvI
3,SfrIHYu8u9c
4,RI4CLyFhUAg
...,...
18647,Vs9QtN8fvho
18648,8yo9GCnM3jc
18649,21aFacRNoWE
18650,xYohdxG_izE


### process 죽이기

In [156]:
import pymysql

# MySQL 연결 설정
conn = pymysql.connect(
    host='project-db-cgi.smhrd.com',
    port=3307,
    user='ehroqkfdbcah',
    password='q1w2e3!@#',
    db='ehroqkfdbcah',
    charset='utf8'
)

# MySQL 커서 생성
cursor = conn.cursor()

# 특정 상태의 모든 프로세스 ID를 가져오는 쿼리
query = "SHOW PROCESSLIST"
cursor.execute(query)
processes = cursor.fetchall()

# 'Sleep' 상태의 프로세스 ID를 수집
sleep_process_ids = [process[0] for process in processes if process[4] == 'Sleep']

# 각 'Sleep' 상태의 프로세스를 종료
for pid in sleep_process_ids:
    try:
        cursor.execute(f"KILL {pid};")
    except pymysql.MySQLError as e:
        print(f"Error killing process {pid}: {e}")

# 변경 사항 커밋
conn.commit()

# MySQL 연결 종료
conn.close()

In [81]:
import pymysql

# MySQL 연결 설정
conn = pymysql.connect(
    host='project-db-cgi.smhrd.com',
    port=3307,
    user='ehroqkfdbcah',
    password='q1w2e3!@#',
    db='ehroqkfdbcah',
    charset='utf8'
)

# MySQL 커서 생성
cursor = conn.cursor()

# 현재 연결된 프로세스 확인
cursor.execute("SHOW PROCESSLIST;")
processes = cursor.fetchall()

# 종료할 프로세스 ID 목록 (필요한 경우 업데이트)
process_ids = [
    304339, 304362, 304363, 304387, 304388,
    304412, 304413, 304459, 304476, 304477
]

# 각 프로세스를 종료
for process in processes:
    pid = process[0]
    if pid in process_ids:
        try:
            cursor.execute(f"KILL {pid};")
            print(f"Successfully killed process {pid}")
        except pymysql.MySQLError as e:
            print(f"Error killing process {pid}: {e}")

# 변경 사항 커밋
conn.commit()

# `iframe` 열 삭제
try:
    cursor.execute("ALTER TABLE `ehroqkfdbcah`.`tb_music` DROP COLUMN `iframe`;")
    conn.commit()
    print("Successfully dropped the 'iframe' column.")
except pymysql.MySQLError as e:
    print(f"Error dropping 'iframe' column: {e}")

# MySQL 연결 종료
conn.close()


Error dropping 'iframe' column: (1091, "Can't DROP 'iframe'; check that column/key exists")


In [83]:
import pymysql

# MySQL 연결 설정
conn = pymysql.connect(
    host='project-db-cgi.smhrd.com',
    port=3307,
    user='ehroqkfdbcah',
    password='q1w2e3!@#',
    db='ehroqkfdbcah',
    charset='utf8'
)

# MySQL 커서 생성
cursor = conn.cursor()

# `iframe` 컬럼 생성
try:
    cursor.execute("ALTER TABLE tb_music ADD COLUMN iframe VARCHAR(256) DEFAULT NULL;")
    conn.commit()
    print("Successfully added the 'iframe' column.")
except pymysql.MySQLError as e:
    print(f"Error adding 'iframe' column: {e}")

# MySQL 연결 종료
conn.close()


Successfully added the 'iframe' column.


In [147]:
import pandas as pd
import pymysql
from tqdm import tqdm

# CSV 파일 경로
csv_file_path = "iframe_link_full_전처리.csv"

# CSV 파일 읽기
df = pd.read_csv(csv_file_path)

# 필요한 데이터 추출
you_df = you_df[['youtube']]

# MySQL 연결 설정
conn = pymysql.connect(
    host='project-db-cgi.smhrd.com',
    port=3307,
    user='ehroqkfdbcah',
    password='q1w2e3!@#',
    db='ehroqkfdbcah',
    charset='utf8'
)

# MySQL 커서 생성
cursor = conn.cursor()

# 데이터프레임을 반복하면서 업데이트 쿼리 실행
for index, row in tqdm(you_df.iterrows(), total=you_df.shape[0]):
    music_id = index + 1  # 인덱스를 musicId로 사용 (1부터 시작)
    iframe_value = row['youtube']  # 'youtube' 열의 값
    try:
        cursor.execute("UPDATE tb_music SET iframe = %s WHERE musicIdx = %s;", (iframe_value, music_id))
        # 각 쿼리 실행 후 커밋
        conn.commit()
    except pymysql.MySQLError as e:
        print(f"Error updating iframe value for musicId {music_id}: {e}")

# MySQL 연결 종료
conn.close()


100%|██████████████████████████████████████████████████████████████████████████| 18652/18652 [2:18:49<00:00,  2.24it/s]


In [150]:
import pymysql

# MySQL 연결 설정
conn = pymysql.connect(
    host='project-db-cgi.smhrd.com',
    port=3307,
    user='ehroqkfdbcah',
    password='q1w2e3!@#',
    db='ehroqkfdbcah',
    charset='utf8',
    cursorclass=pymysql.cursors.DictCursor  # DictCursor 설정 추가
)

try:
    with conn.cursor() as cursor:
        # 중복된 행의 개수를 계산하는 SQL 쿼리
        count_duplicates_sql = """
        SELECT title, artist, COUNT(*) as count
        FROM tb_music
        GROUP BY title, artist
        HAVING COUNT(*) > 1;
        """

        cursor.execute(count_duplicates_sql)
        duplicate_counts = cursor.fetchall()

        # 중복된 행의 총 개수 계산
        total_duplicates = sum(row['count'] - 1 for row in duplicate_counts)  # 중복된 행의 개수는 count - 1 의 합계
        print(f"Total number of duplicate rows: {total_duplicates}")
finally:
    conn.close()

Total number of duplicate rows: 7572


In [153]:
import pymysql

# MySQL 연결 설정
conn = pymysql.connect(
    host='project-db-cgi.smhrd.com',
    port=3307,
    user='ehroqkfdbcah',
    password='q1w2e3!@#',
    db='ehroqkfdbcah',
    charset='utf8',
    cursorclass=pymysql.cursors.DictCursor  # DictCursor 설정 추가
)

try:
    with conn.cursor() as cursor:
        # 중복된 행의 개수를 계산하는 SQL 쿼리
        count_duplicates_sql = """
        SELECT title, artist, COUNT(*) as count
        FROM tb_music
        GROUP BY title, artist
        HAVING COUNT(*) > 1;
        """

        cursor.execute(count_duplicates_sql)
        duplicate_counts = cursor.fetchall()

        # 중복된 행의 총 개수 계산
        total_duplicates = sum(row['count'] - 1 for row in duplicate_counts)  # 중복된 행의 개수는 count - 1 의 합계
        print(f"Total number of duplicate rows: {total_duplicates}")

        # 중복된 행 삭제하는 SQL 쿼리
        delete_duplicates_sql = """
        DELETE t1 FROM tb_music t1
        INNER JOIN (
            SELECT MIN(musicIdx) as minId, title, artist
            FROM tb_music
            GROUP BY title, artist
            HAVING COUNT(*) > 1
        ) t2 
        ON t1.title = t2.title AND t1.artist = t2.artist
        WHERE t1.musicIdx > t2.minId;
        """

        cursor.execute(delete_duplicates_sql)
        conn.commit()
finally:
    conn.close()

Total number of duplicate rows: 7572


In [155]:
import pymysql

# MySQL 연결 설정
conn = pymysql.connect(
    host='project-db-cgi.smhrd.com',
    port=3307,
    user='ehroqkfdbcah',
    password='q1w2e3!@#',
    db='ehroqkfdbcah',
    charset='utf8',
    cursorclass=pymysql.cursors.DictCursor
)

try:
    with conn.cursor() as cursor:
        # 특정 컬럼 삭제
        drop_column_sql = "ALTER TABLE tb_music DROP COLUMN tempIdx;"
        cursor.execute(drop_column_sql)
        conn.commit()

finally:
    conn.close()

print("특정 컬럼 삭제 완료")

특정 컬럼 삭제 완료


In [ ]:
import pymysql
from tqdm import tqdm

# MySQL 연결 설정
conn = pymysql.connect(
    host='project-db-cgi.smhrd.com',
    port=3307,
    user='ehroqkfdbcah',
    password='q1w2e3!@#',
    db='ehroqkfdbcah',
    charset='utf8',
    cursorclass=pymysql.cursors.DictCursor
)

try:
    with conn.cursor() as cursor:
        # 1. 임시 컬럼 추가
        cursor.execute("ALTER TABLE tb_music ADD COLUMN tempIdx INT;")
        conn.commit()

        # 2. 기존 테이블의 데이터를 가져오고, tempIdx를 순차적으로 할당
        select_sql = "SELECT musicIdx FROM tb_music ORDER BY musicIdx;"
        cursor.execute(select_sql)
        rows = cursor.fetchall()

        new_musicIdx = 1
        musicIdx_mapping = {}

        for row in tqdm(rows, desc="Assigning new tempIdx values"):
            old_musicIdx = row['musicIdx']
            musicIdx_mapping[old_musicIdx] = new_musicIdx
            update_sql = "UPDATE tb_music SET tempIdx = %s WHERE musicIdx = %s;"
            cursor.execute(update_sql, (new_musicIdx, old_musicIdx))
            new_musicIdx += 1

        conn.commit()

finally:
    conn.close()

print("1단계 완료: 임시 컬럼 추가 및 tempIdx 값 할당 완료")

In [ ]:
import pymysql
from tqdm import tqdm

# MySQL 연결 설정
conn = pymysql.connect(
    host='project-db-cgi.smhrd.com',
    port=3307,
    user='ehroqkfdbcah',
    password='q1w2e3!@#',
    db='ehroqkfdbcah',
    charset='utf8',
    cursorclass=pymysql.cursors.DictCursor
)

try:
    with conn.cursor() as cursor:
        # 이전 단계에서 생성한 musicIdx_mapping을 다시 가져와야 합니다
        # 이를 위해 파일에 저장하거나 데이터베이스에 저장할 수 있습니다
        # 여기서는 간단하게 다시 생성한다고 가정합니다
        
        # 1. 기존 테이블의 데이터를 가져오고, tempIdx를 순차적으로 할당
        select_sql = "SELECT musicIdx, tempIdx FROM tb_music ORDER BY musicIdx;"
        cursor.execute(select_sql)
        rows = cursor.fetchall()

        musicIdx_mapping = {row['musicIdx']: row['tempIdx'] for row in rows}

        # 2. 외래 키를 업데이트하는 쿼리 생성
        update_foreign_keys_sql = "UPDATE tb_aiplaylist SET musicIdx = CASE "
        for old_musicIdx, new_musicIdx in tqdm(musicIdx_mapping.items(), desc="Updating foreign keys"):
            update_foreign_keys_sql += f"WHEN musicIdx = {old_musicIdx} THEN {new_musicIdx} "
        update_foreign_keys_sql += "END WHERE musicIdx IN ("
        update_foreign_keys_sql += ",".join(str(old_musicIdx) for old_musicIdx in musicIdx_mapping.keys())
        update_foreign_keys_sql += ");"

        cursor.execute(update_foreign_keys_sql)
        conn.commit()

finally:
    conn.close()

print("2단계 완료: 외래 키 업데이트 완료")

In [ ]:
import pymysql
from tqdm import tqdm

# MySQL 연결 설정
conn = pymysql.connect(
    host='project-db-cgi.smhrd.com',
    port=3307,
    user='ehroqkfdbcah',
    password='q1w2e3!@#',
    db='ehroqkfdbcah',
    charset='utf8',
    cursorclass=pymysql.cursors.DictCursor
)

try:
    with conn.cursor() as cursor:
        # 1. 기존 musicIdx를 tempIdx 값으로 업데이트
        select_sql = "SELECT musicIdx FROM tb_music ORDER BY musicIdx;"
        cursor.execute(select_sql)
        rows = cursor.fetchall()

        for row in tqdm(rows, desc="Updating musicIdx to tempIdx"):
            update_sql = "UPDATE tb_music SET musicIdx = tempIdx WHERE musicIdx = %s;"
            cursor.execute(update_sql, (row['musicIdx'],))

        conn.commit()

        # 2. 임시 컬럼 제거
        cursor.execute("ALTER TABLE tb_music DROP COLUMN tempIdx;")
        conn.commit()

        # 3. AUTO_INCREMENT 속성을 재설정하여 musicIdx가 순차적으로 증가하도록 설정
        cursor.execute("ALTER TABLE tb_music AUTO_INCREMENT = 1;")
        conn.commit()

finally:
    conn.close()

print("3단계 완료: musicIdx 업데이트 및 임시 컬럼 제거 완료")

In [ ]:
import pymysql
from tqdm import tqdm

# MySQL 연결 설정
conn = pymysql.connect(
    host='project-db-cgi.smhrd.com',
    port=3307,
    user='ehroqkfdbcah',
    password='q1w2e3!@#',
    db='ehroqkfdbcah',
    charset='utf8',
    cursorclass=pymysql.cursors.DictCursor
)